In [7]:
import tensorflow as tf
import numpy as np
import tensorflow_datasets as tfds
import math

import PIL.Image
import IPython.display as display
import matplotlib.pyplot as plt
import matplotlib as mpl

import time
import functools
# for looking at files
import glob
import tqdm
import datetime

from keras.models import load_model
from keras.models import Sequential
from keras.layers import Dense, Flatten

### Load Data

In [8]:
style_to_number = {
    'cubism': 0,
    'expressionism' : 1,
    'romanticism' : 2
}

def load_images(folder):
        '''Load images and create corresponding numerical labels for the classes
    Input: 
            path: a path
            
    Output:
            images: a list of arrays 
            labels: a list of numerical labels'''
    
    
    # load all path for the images, respectively to their epoch
    path = '../input/art-movements/dataset/' + folder
    
    cubism_paths = glob.glob(path + '/cubism/*')
    expressionism_paths = glob.glob(path + '/expressionism/*')
    romanticism_paths = glob.glob(path + '/romanticism/*')
    
    combined_paths = [cubism_paths, expressionism_paths, romanticism_paths]
    
    images = []
    labels = []

    # load images and create art-style-corresponding label list for them
    for i, art_style in enumerate(combined_paths):
        for image_path in art_style:
            image = np.asarray(tf.keras.preprocessing.image.load_img(image_path))     
            images.append(image)
            labels.append(i)

    return images, labels


def resize_images(images):
    '''Images get resized into a uniform size'''
    
    return [tf.image.resize(image, [128,128]) for image in images]


def crop_images(images):
    '''Crop the image in the biggest possible square
        
            Input: Array of images
            
            Output: Array if square image'''
    
    cropped = []
    
    for image in images:
        shape = np.min(image.shape[:-1])
        cropped_image = tf.image.resize_with_crop_or_pad(image, shape, shape)
        cropped.append(cropped_image)

    return cropped


# we did not use it, but it could be used to create more examples for training the network. 
# It is questionable if the style may gets distorted too much
def random_crop(images):
    '''Randomly crop all images into a uiform size'''
    return [tf.image.random_crop(image, size=[128, 128, 3]) for image in images]

In [9]:
# create arrays of labels ad images
train_images, train_labels = load_images(folder='train')
test_images, test_labels = load_images(folder = 'test')

In [10]:
# crop and resize images into a uniform shape to be able to create a tensorflow dataset
train_images_cropped = crop_images(train_images)
train_images_resized = resize_images(train_images_cropped)
test_images_cropped = crop_images(test_images)
test_images_resized = resize_images(test_images_cropped)

### Create Datasets 

In [48]:
# Generator do merge the images and labels for train and test-datasets
def train_data_gen():
    for i, image in enumerate(train_images):
        yield image, train_labels[i]

def test_data_gen():
    for i, image in enumerate(test_images):
        yield image, test_labels[i]


# creste a tf datasets from the loaded images and the labels
train_ds = tf.data.Dataset.from_generator(train_data_gen, output_signature=(tf.TensorSpec(shape=(None, None, 3)),
                                                             tf.TensorSpec(shape=(), dtype=tf.int32))
                                                             )

test_ds = tf.data.Dataset.from_generator(test_data_gen, output_signature=(tf.TensorSpec(shape=(None, None, 3)),
                                                             tf.TensorSpec(shape=(), dtype=tf.int32))
                                                             )

#### Plottting some Images

In [45]:
plt.imshow(2*(train_images[1]/256)-1) 

In [41]:
plt.imshow(train_images_cropped[1]) 

In [40]:
plt.imshow(2*(train_images_resized[1]/256)-1) 

### Data Augmentation

In [12]:
# create the generator we 
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255, 
    #rotation_range=40,
    brightness_range=(0.5,1.5),
    #width_shift_range=0.15,
    #height_shift_range=0.15,
    shear_range=5,
    horizontal_flip=True,
    vertical_flip = True,
    zoom_range = [0.7, 1]
)

# create an iterator using datagen.flow
train_images_resized = np.asarray(train_images_resized)
train_labels = np.asarray(train_labels)
train_generator = datagen.flow(train_images_resized, train_labels, batch_size=64)


def generator(num_batches):
    for i, train_tuple in enumerate(train_generator):
        yield train_tuple
        if i >= num_batches:
            return

### Plott Augmented Data

In [ ]:
###Plotting Augmented Images

# plot images of first batch
fig, ax = plt.subplots(4,8,figsize=(20,10))
fig.tight_layout()
ax = ax.flatten()
for img_b, label_b in train_generator:
    for i in range(32):   
        img = img_b[i]
        l = label_b[i]
        
        ax[i].imshow(img)  
        ax[i].set_title((img.shape))
        ax[i].axis("off")

    break   

fig, ax = plt.subplots(4,8,figsize=(20,10))
fig.tight_layout()
ax = ax.flatten()
for img_b, label_b in train_generator:
    for i in range(32):   
        img = img_b[i]
        l = label_b[i]
        ax[i].imshow(img)  
        ax[i].set_title((img.shape))
        ax[i].axis("off")

    break

### Preprocess data

In [13]:
def augmented_data_creator(data):
    num_batches = 500
    # pass generator, outputtypes and num_batches
    # args needs to be tuple of tensors
    augmented_data = tf.data.Dataset.from_generator(generator, (tf.float32, tf.float32), args=(tf.constant(num_batches),))
    # Now do the remaining tensorflow pipeline
    augmented_data = augmented_data.map(lambda x, y: (x, tf.one_hot(tf.cast(y, tf.uint8), 3)))
    augmented_data = augmented_data.map(lambda x,y: ((2*x-1), y))

    return augmented_data


# data pipeline to pre-process the images
def preprocessing_data(data):
    'preprocesses the dataset'
    #convert data from uint8 to float32
    data = data.map(lambda img, target: (tf.cast(img, tf.float32), target))
    data = data.map(lambda img, target: (tf.image.resize(img, [128,128]), target))
    data = data.map(lambda img, target: ((img/128.)-1., target))
    data = data.map(lambda img, target: (img, tf.one_hot(target, depth=3)))
    data = data.batch(64)

    return data 

In [14]:
#concatenate the original data and the augmented data, or only use the unaugmented data to compare training progress. Then cache, shuffle, prefetch
#train_data = train_ds.apply(preprocessing_data)
#unaugmented_train_data = train_data.cache().shuffle(64).prefetch(20)
#train_data = train_data.concatenate(augmented_data).cache().shuffle(64).prefetch(20)

# We only used the augmented data in the End
augmented_data = augmented_data_creator(train_ds)
train_data = augmented_data.cache().shuffle(64).prefetch(20)
test_data = test_ds.apply(preprocessing_data).cache().shuffle(64).prefetch(20)

### Pretrained ResNetV2

In [ ]:
# load a pretraines ResV2
pretrained_resv2 = tf.keras.applications.resnet_v2.ResNet101V2(include_top = False)

## Freezing all earlier layers that represent low-level features
for layer in pretrained_resv2.layers[:128]:
    layer.trainable = False

# turn the model into a sequential to add layers for our need
tuning_model = Sequential()
tuning_model.add(pretrained_resv2)
tuning_model.add(tf.keras.layers.GlobalAveragePooling2D())
tuning_model.add(Dense(256, 'relu'))
tuning_model.add(Dense(3, 'softmax'))

tuning_model.compile(loss='categorical_crossentropy', optimizer = tf.keras.optimizers.Adam(learning_rate = 1e-4), metrics=['acc'])

### Pretrained V3

In [ ]:
# using the same pre-trained weights as suggested by another tutorial with the art movement data set
v3_weights = '../input/keras-pretrained-models/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
pretrained_v3 = tf.keras.applications.InceptionV3(input_shape = (128,128,3), include_top = False, weights = v3_weights)

## Freezing all earlier layers that represent low-level features
for layer in pretrained_v3.layers[:64]:
    layer.trainable = False

# turn the model into a sequential to add layers for our need
tuning_model = Sequential()
tuning_model.add(pretrained_v3)
tuning_model.add(tf.keras.layers.GlobalAveragePooling2D())
tuning_model.add(Dense(256, 'relu'))
tuning_model.add(Dense(3, 'softmax'))

tuning_model.compile(loss='categorical_crossentropy', optimizer = tf.keras.optimizers.Adam(learning_rate = 1e-4), metrics=['acc'])

### Pretrained VGG19

In [ ]:
#trying a VGG19
pretrained_vgg19 = tf.keras.applications.vgg19.VGG19(input_shape = (128,128,3), include_top = False)

## Freezing all earlier layers that represent low-level features
for layer in pretrained_vgg19.layers[:18]:
    layer.trainable = False
    
# turn the model into a sequential to add layers for our need
tuning_model = Sequential()
tuning_model.add(pretrained_vgg19)
tuning_model.add(tf.keras.layers.GlobalAveragePooling2D())
tuning_model.add(Dense(256, 'relu'))
tuning_model.add(Dense(3, 'softmax'))

tuning_model.compile(loss='categorical_crossentropy', optimizer = tf.keras.optimizers.Adam(learning_rate = 1e-4), metrics=['acc'])

### Pretrained ResNet50 - Simpler Architecture

In [ ]:
# trying a pre-trained ResNet50
pretrained_res = tf.keras.applications.resnet50.ResNet50(include_top=False)

## Freezing all earlier layers that represent low-level features
for layer in pretrained_res.layers[:32]:
    layer.trainable = False

# turn the model into a sequential to add layers for our need
tuning_model = Sequential()
tuning_model.add(pretrained_res)
tuning_model.add(tf.keras.layers.GlobalAveragePooling2D())
tuning_model.add(Dense(256, 'relu'))
tuning_model.add(Dense(3, 'softmax'))

tuning_model.compile(loss='categorical_crossentropy', optimizer = tf.keras.optimizers.Adam(learning_rate = 1e-4), metrics=['acc'])

In [ ]:
tuning_model.summary()
# load previously saved weights to continue training. 
#tuning_model.load_weights(f"saved_model_artstyle_discrimination_network{hyperparameter_string_res}")

In [15]:
def train_step(model, input, target, loss_function, optimizer):
    # loss_object and optimizer_object are instances of respective tensorflow classes
    with tf.GradientTape() as tape:
        prediction = model(input)#, train = True )
        loss = loss_function(target, prediction)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss


def test(model, test_data, loss_function):
    # test over complete test data

    test_accuracy_aggregator = []
    test_loss_aggregator = []

    for (input, target) in test_data:
        prediction = model(input)#, train = False)
        sample_test_loss = loss_function(target, prediction)
        sample_test_accuracy = np.argmax(target, axis=1) == np.argmax(prediction, axis=1)
        sample_test_accuracy = np.mean(sample_test_accuracy)
        test_loss_aggregator.append(sample_test_loss.numpy())
        test_accuracy_aggregator.append(sample_test_accuracy)

    test_loss = tf.reduce_mean(test_loss_aggregator)
    test_accuracy = tf.reduce_mean(test_accuracy_aggregator)

    return test_loss, test_accuracy

In [ ]:
# load tensorboard extension
%load_ext tensorboard

# clear all previous logs:
!rm -rf ./logs/

# define file-path for log file
file_path = "final_project/discrimination_network"

# Define where to save the log
hyperparameter_string = "Adam_LR000025_resv2_Layersfrozen_32_cache"
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

train_log_path = f"logs/{file_path}/{hyperparameter_string}/{current_time}/train"
test_log_path = f"logs/{file_path}/{hyperparameter_string}/{current_time}/test"

# log writer for training metrics
train_summary_writer = tf.summary.create_file_writer(train_log_path)

# log writer for validation metrics
test_summary_writer = tf.summary.create_file_writer(test_log_path)

In [ ]:
# Baseline model for comparison
tf.keras.backend.clear_session()
#%tensorboard --logdir logs/

#assign train and test dataset
train_dataset = train_data
test_dataset = test_data

# for training with the unaugmented images in the training set included
#train_dataset = unaugmented_train_data

### Hyperparameter ################################################################################
num_epochs = 40
learning_rate = 25e-6

# Assign the model.
model = tuning_model

# Initialize the loss, categorical cross entropy
cross_entropy_loss = tf.keras.losses.CategoricalCrossentropy()
# Initialize the optimizer Adam, only adjusting the learning-rate
optimizer = tf.keras.optimizers.Adam(learning_rate)

# Initialize lists for later visualization.
train_losses = []
train_accuracies = []

test_losses = []
test_accuracies = []


#testing on the validation dataset once before we begin
test_loss, test_accuracy = test(model, test_dataset, cross_entropy_loss)
test_losses.append(test_loss)
test_accuracies.append(test_accuracy)

#check how model performs on train data once before we begin
train_loss, train_accuracy = test(model, train_dataset, cross_entropy_loss)
train_losses.append(train_loss)
train_accuracies.append(train_accuracy)

print(f'Untrained Accuracy on Train Data {train_accuracies[-1]}')


# We train for num_epochs epochs or until a certain accuracy for unseen data is met
for epoch in range(num_epochs):
    print(f'Epoch: {str(epoch)} starting with accuracy {test_accuracies[-1]}')

    #training (and checking in with training)
    epoch_loss_agg = []
    for input,target in train_dataset:

        #randomly crop images while training to enhance the amount of data
        images = []
        for img in input:
            cropsize = 256
            if np.min(img.shape[:-1]) < cropsize:
                cropsize = np.min(img.shape[:-1])
            img = tf.image.resize_with_crop_or_pad(img, cropsize, cropsize)
            img = tf.image.resize(img, [128,128])
            images.append(img)        
        
        images = tf.convert_to_tensor(images)

        train_loss = train_step(model, images, target, cross_entropy_loss, optimizer)
        epoch_loss_agg.append(train_loss)

        
    # track training loss
    train_losses.append(tf.reduce_mean(epoch_loss_agg))
    
    train_loss , train_accuracy = test(model, train_dataset, cross_entropy_loss)
    train_accuracies.append(train_accuracy)
    mean_train_loss = np.mean(epoch_loss_agg)
    
    print(f'Epoch: {str(epoch)} ending with accuracy on Training Set {train_accuracies[-1]}')

    with train_summary_writer.as_default():
        tf.summary.scalar(f"Train Accuracy", train_accuracy, step=epoch)

    # testing, so we can track accuracy and test loss
    test_loss, test_accuracy = test(model, test_dataset, cross_entropy_loss)
    test_losses.append(test_loss)
    test_accuracies.append(test_accuracy)
    
    # to prevent excessive training, we stop the model, once the desired test-accuracy is reached
    if test_accuracy > 0.85:
        break

### Saving the Model and the weights

In [38]:
print(hyperparameter_string)

In [ ]:
hyperparameter_string = "Adam_LR000025_resv2_Layersfrozen_128_cache_ACC85"
tuning_model.save_weights(f"saved_model_artstyle_discrimination_network{hyperparameter_string}", save_format="tf")

In [ ]:
#tuning_model.trainable = False
#tuning_model.save("art_transfer_discrimination_model_resV2_Adam_LR000025_frozenlayers128_ACC85.h5")

## Using the trained Model to see how our Art-style-Transfer Performs objective

In [17]:
art_transfer_discrimination_model = tf.keras.models.load_model("../input/art-transfer-discrimination-model-resv2/art_transfer_discrimination_model_resV2_Adam_LR000025_frozenlayers128_ACC85.h5")

In [189]:
art_transfer_discrimination_model.summary()

### Loading the pictures

In [62]:
# dictionary for possible later translations
style_to_number = {
    'cubism': 0,
    'expressionism' : 1,
    'romanticism' : 2
}


def load_images(path):
    '''Load images and create corresponding numerical labels for the classes
    Input: 
            path: a path
            
    Output:
            images: a list of arrays 
            labels: a list of numerical labels'''
    
    
    # load all path for the images, respectively to their epoch
    cubism_paths = glob.glob(path + '/cubism/*')
    expressionism_paths = glob.glob(path + '/expressionism/*')
    romanticism_paths = glob.glob(path + '/romanticism/*')

    combined_paths = [cubism_paths, expressionism_paths, romanticism_paths]
    images = []
    labels = []
    
    # load images and create art-style-corresponding label list for them
    for i, art_style in enumerate(combined_paths):
        for image_path in art_style:
            image = np.asarray(tf.keras.preprocessing.image.load_img(image_path))     
            images.append(image)
            labels.append(i)

    return images, labels


# Generators to merge the labels and images from the datasets
def vgg_data_gen():
    for i, image in enumerate(vgg_images):
        yield image, vgg_labels[i]

        
def res_data_gen():
    for i, image in enumerate(res_images):
        yield image, res_labels[i]

        
def vgg_fn_data_gen():
    for i, image in enumerate(vgg_fn_images):
        yield image, vgg_fn_labels[i]

### Determine Performence

In [57]:
# define the entropy
cross_entropy_loss = tf.keras.losses.CategoricalCrossentropy()

# load the dataset from the VGG with the content image as input
vgg_images, vgg_labels = load_images(path = '../input/vgg19-augmented-from-content-image/VGG19_augmented_from_content_image')

# create the dataset from images and labels
vgg_ds = tf.data.Dataset.from_generator(vgg_data_gen, output_signature=(tf.TensorSpec(shape=(None, None, 3)),
                                                             tf.TensorSpec(shape=(), dtype=tf.int32)))

# preprocessing
vgg_data = vgg_ds.apply(preprocessing_data).cache().shuffle(64).prefetch(20)

In [58]:
# load the dataset from the VGG with the noise image as input
vgg_fn_images, vgg_fn_labels = load_images(path = '../input/vgg19-augmented-from-noise/VGG19_augmented_from_noise')

# create the dataset from images and labels
vgg_fn_ds = tf.data.Dataset.from_generator(vgg_fn_data_gen, output_signature=(tf.TensorSpec(shape=(None, None, 3)),
                                                             tf.TensorSpec(shape=(), dtype=tf.int32)))

# preprocessing
vgg_fn_data = vgg_ds.apply(preprocessing_data).cache().shuffle(64).prefetch(20)

In [36]:
# load the dataset from the ResNet
res_images, res_labels = load_images(path = '../input/resnet-style-transferred-images/Results Pretrained ResNet50')

# create the dataset from images and labels
res_ds = tf.data.Dataset.from_generator(res_data_gen, output_signature=(tf.TensorSpec(shape=(None, None, 3)),
                                                             tf.TensorSpec(shape=(), dtype=tf.int32)))

# preprocessing
res_data = res_ds.apply(preprocessing_data).cache().shuffle(64).prefetch(20)

#### Peak at the data

In [59]:
#peek at the images and labels
print(plt.imshow(vgg_fn_images[1]), vgg_fn_labels[1])

In [60]:
#peek at the images and labels
print(plt.imshow(vgg_images[1]), vgg_labels[1])

In [204]:
#peek at the images and labels
print(plt.imshow(res_images[1]), res_labels[1])

### Calculate the Accuracy of the Network with different Style Transferred Images

In [61]:
#calculate the performence of the network on differnt style-transferred images:
#VGG19, content image as input
vgg_loss, vgg_accuracy = test(art_transfer_discrimination_model, vgg_data, cross_entropy_loss)
print(f'VGG19 Accuracy with Content Image as Input: {vgg_accuracy}.')

#VGG19, noise image as input
vgg_fn_loss, vgg_fn_accuracy = test(art_transfer_discrimination_model, vgg_fn_data, cross_entropy_loss)
print(f'VGG19 Accuracy with Noise Image as Input: {vgg_fn_accuracy}.')

#ResNet50, content image as input
res_loss, res_accuracy = test(art_transfer_discrimination_model, res_data, cross_entropy_loss)
print(f'Res Accuracy with Content Image as Input: {res_accuracy}.')